<a href="https://colab.research.google.com/github/Vishal172-blip/Certificates_Link/blob/main/Assigement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:


# Conversation history will be stored here as a list of dictionaries
conversation_history = []

# Counter to keep track of how many user-assistant exchanges (runs) have occurred
run_count = 0

def add_message(role: str, content: str):
    """
    Add a message to the conversation history.
    role: 'user' or 'assistant'
    content: message text
    """
    global run_count
    if role == "assistant":
        # Increment run count each time assistant responds (marks a full exchange)
        run_count += 1

    message = {
        "role": role,
        "content": content
    }
    conversation_history.append(message)

def get_history():
    """Return the full conversation history as a list of messages."""
    return conversation_history

# ---- Demo: Adding some sample messages ----

add_message("user", "Hi there!")
add_message("assistant", "Hello! How can I help you?")
add_message("user", "Can you tell me about Groq APIs?")
add_message("assistant", "Sure! Groq provides OpenAI-compatible APIs for AI/ML tasks.")

# Print the current conversation history
for idx, msg in enumerate(get_history(), start=1):
    print(f"{idx}. {msg['role'].capitalize()}: {msg['content']}")

# Print run count to show how many assistant replies so far
print("\nTotal conversation runs so far:", run_count)





1. User: Hi there!
2. Assistant: Hello! How can I help you?
3. User: Can you tell me about Groq APIs?
4. Assistant: Sure! Groq provides OpenAI-compatible APIs for AI/ML tasks.

Total conversation runs so far: 2


In [30]:


# Reuse the conversation_history and run_count from Part 1
# Assuming you already ran Part 1 code in the same notebook.

def truncate_history_by_turns(n: int):
    """
    Return the last n messages from the conversation history.
    Does not modify the original history.
    """
    return conversation_history[-n:] if n < len(conversation_history) else conversation_history.copy()

def truncate_history_by_length(max_chars: int):
    """
    Return a truncated copy of the conversation history
    limited by total character length.
    Iterates from newest to oldest and stops when limit is reached.
    """
    truncated = []
    total_chars = 0
    # Iterate in reverse (newest first)
    for msg in reversed(conversation_history):
        length = len(msg['content'])
        if total_chars + length > max_chars:
            break
        truncated.insert(0, msg)  # insert at start to maintain order
        total_chars += length
    return truncated

# ---- Demo: Show truncation ----

print("=== Full History ===")
for i, m in enumerate(conversation_history, 1):
    print(f"{i}. {m['role'].capitalize()}: {m['content']}")

# Truncate by last 3 messages
print("\n=== Truncate: Last 3 Messages ===")
for i, m in enumerate(truncate_history_by_turns(3), 1):
    print(f"{i}. {m['role'].capitalize()}: {m['content']}")

# Truncate by max 50 characters
print("\n=== Truncate: Max 50 Characters ===")
for i, m in enumerate(truncate_history_by_length(50), 1):
    print(f"{i}. {m['role'].capitalize()}: {m['content']}")


=== Full History ===
1. User: Hi there!
2. Assistant: Hello! How can I help you?
3. User: Can you tell me about Groq APIs?
4. Assistant: Sure! Groq provides OpenAI-compatible APIs for AI/ML tasks.

=== Truncate: Last 3 Messages ===
1. Assistant: Hello! How can I help you?
2. User: Can you tell me about Groq APIs?
3. Assistant: Sure! Groq provides OpenAI-compatible APIs for AI/ML tasks.

=== Truncate: Max 50 Characters ===


In [32]:

import pandas as pd

conversation_data = [
    {"user_message": "Hi", "assistant_response": "Hello!"},
    {"user_message": "What's the price?", "assistant_response": "It's $10."},
    {"user_message": "Thanks!", "assistant_response": "You're welcome!"}
]

df = pd.DataFrame(conversation_data)
df.to_csv("conversation_logs.csv", index=False)
print("conversation_logs.csv saved successfully!")


conversation_logs.csv saved successfully!


In [33]:

# Process and analyze the stored conversational data.

import pandas as pd
from collections import Counter

# Load the stored conversation logs
df = pd.read_csv("conversation_logs.csv")

# --- Basic cleaning ---
# Strip whitespace and drop any completely empty messages
df['user_message'] = df['user_message'].str.strip()
df['assistant_response'] = df['assistant_response'].str.strip()
df.dropna(subset=['user_message', 'assistant_response'], inplace=True)

# --- Basic statistics ---
# Count total conversations
total_conversations = df.shape[0]

# Count average message length for user and assistant
avg_user_len = df['user_message'].apply(len).mean()
avg_assistant_len = df['assistant_response'].apply(len).mean()

# Identify most common words in user messages
all_words = " ".join(df['user_message']).lower().split()
common_words = Counter(all_words).most_common(10)

# --- Filter conversations by keyword (example) ---
def filter_by_keyword(keyword):
    return df[df['user_message'].str.contains(keyword, case=False)]

# --- Print summary ---
print(f"Total conversations stored: {total_conversations}")
print(f"Average user message length: {avg_user_len:.2f} characters")
print(f"Average assistant response length: {avg_assistant_len:.2f} characters\n")

print("Top 10 most common words in user messages:")
for word, count in common_words:
    print(f"{word}: {count}")

# Example filter usage:
print("\nSample conversations mentioning 'price':")
print(filter_by_keyword('price').head())


Total conversations stored: 3
Average user message length: 8.67 characters
Average assistant response length: 10.00 characters

Top 10 most common words in user messages:
hi: 1
what's: 1
the: 1
price?: 1
thanks!: 1

Sample conversations mentioning 'price':
        user_message assistant_response
1  What's the price?          It's $10.


In [34]:

import pandas as pd
from collections import Counter

# ---------- Load & Analyze Data ----------
df = pd.read_csv("conversation_logs.csv")
df['user_message'] = df['user_message'].str.strip()
df['assistant_response'] = df['assistant_response'].str.strip()
df.dropna(subset=['user_message','assistant_response'], inplace=True)

total_conversations = df.shape[0]
avg_user_len = df['user_message'].apply(len).mean()
avg_assistant_len = df['assistant_response'].apply(len).mean()
all_words = " ".join(df['user_message']).lower().split()
common_words = Counter(all_words).most_common(10)

# ---------- Truncate History ----------
def truncate_history_by_turns(history, n):
    return history[-n:]

conversation_history = list(
    df[['user_message','assistant_response']].itertuples(index=False, name=None)
)
truncated = truncate_history_by_turns(conversation_history, 5)

# ---------- Fake Summarization ----------
history_text = "\n".join([f"User: {u}\nAssistant: {a}" for u,a in truncated])
summary = "• User mostly asks about prices.\n• Assistant gives concise answers.\n• Conversation is polite and short.\n• Most common topic is product info."

# ---------- Final Report ----------
print("=== 📊 Stats Report ===")
print(f"Total conversations stored: {total_conversations}")
print(f"Average user message length: {avg_user_len:.2f} characters")
print(f"Average assistant response length: {avg_assistant_len:.2f} characters\n")

print("Top 10 most common words in user messages:")
for word,count in common_words:
    print(f"{word}: {count}")

print("\n=== 📝 Simulated Summary of Recent History ===")
print(summary)


=== 📊 Stats Report ===
Total conversations stored: 3
Average user message length: 8.67 characters
Average assistant response length: 10.00 characters

Top 10 most common words in user messages:
hi: 1
what's: 1
the: 1
price?: 1
thanks!: 1

=== 📝 Simulated Summary of Recent History ===
• User mostly asks about prices.
• Assistant gives concise answers.
• Conversation is polite and short.
• Most common topic is product info.


## Task 2

In [44]:

# JSON Schema Creation for extracting 5 details from chat

import json

# Define the JSON schema
chat_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Name of the user"},
        "email": {"type": "string", "format": "email", "description": "Email of the user"},
        "phone": {"type": "string", "description": "Phone number of the user"},
        "location": {"type": "string", "description": "Location of the user"},
        "age": {"type": "integer", "description": "Age of the user"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

# Print schema nicely
print(json.dumps(chat_schema, indent=4))


{
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "Name of the user"
        },
        "email": {
            "type": "string",
            "format": "email",
            "description": "Email of the user"
        },
        "phone": {
            "type": "string",
            "description": "Phone number of the user"
        },
        "location": {
            "type": "string",
            "description": "Location of the user"
        },
        "age": {
            "type": "integer",
            "description": "Age of the user"
        }
    },
    "required": [
        "name",
        "email",
        "phone",
        "location",
        "age"
    ]
}


In [43]:

# Sample chat parsing using the JSON schema (simulated without real API)

import json

# Sample chats
sample_chats = [
    "Hi, my name is Vishal Kumar. My email is vishal@example.com, phone 9876543210, I live in Delhi and I am 20 years old.",
    "Hello, I'm Priya Sharma, email priya123@mail.com, phone 9123456789, from Mumbai, 22 years old.",
    "Hey, name is Raj, email raj456@gmail.com, phone 9988776655, living in Bangalore, age 25."
]

# Function to simulate extraction using the schema
def extract_info_from_chat(chat_text):
    # This is a simulated parser; in real scenario Groq/OpenAI function would do this
    info = {
        "name": "",
        "email": "",
        "phone": "",
        "location": "",
        "age": 0
    }
    # Simple extraction using splits (basic simulation)
    import re
    name_match = re.search(r"name is (\w+ \w+)|I'm (\w+ \w+)|name is (\w+)", chat_text)
    info['name'] = next((m for m in name_match.groups() if m), "")

    email_match = re.search(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", chat_text)
    info['email'] = email_match.group() if email_match else ""

    phone_match = re.search(r"\b\d{10}\b", chat_text)
    info['phone'] = phone_match.group() if phone_match else ""

    loc_match = re.search(r"live in (\w+)|from (\w+)|living in (\w+)", chat_text)
    info['location'] = next((m for m in loc_match.groups() if m), "")

    age_match = re.search(r"(\d{2}) years old|age (\d{2})", chat_text)
    info['age'] = int(next((m for m in age_match.groups() if m), 0))

    return info

# Parse all sample chats
parsed_outputs = [extract_info_from_chat(chat) for chat in sample_chats]

# Print JSON outputs
for i, output in enumerate(parsed_outputs, 1):
    print(f"--- Chat {i} Parsed ---")
    print(json.dumps(output, indent=4))


--- Chat 1 Parsed ---
{
    "name": "Vishal Kumar",
    "email": "vishal@example.com",
    "phone": "9876543210",
    "location": "Delhi",
    "age": 20
}
--- Chat 2 Parsed ---
{
    "name": "Priya Sharma",
    "email": "priya123@mail.com",
    "phone": "9123456789",
    "location": "Mumbai",
    "age": 22
}
--- Chat 3 Parsed ---
{
    "name": "Raj",
    "email": "raj456@gmail.com",
    "phone": "9988776655",
    "location": "Bangalore",
    "age": 25
}


In [37]:
# --- Task 2: Part 3 ---
# Validate parsed JSON outputs against schema

from jsonschema import validate, ValidationError

# Reuse the schema from Part 1
chat_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Name of the user"},
        "email": {"type": "string", "format": "email", "description": "Email of the user"},
        "phone": {"type": "string", "description": "Phone number of the user"},
        "location": {"type": "string", "description": "Location of the user"},
        "age": {"type": "integer", "description": "Age of the user"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

# Assume parsed_outputs from Part 2
# parsed_outputs = [...]

# Validate each JSON
for i, output in enumerate(parsed_outputs, 1):
    try:
        validate(instance=output, schema=chat_schema)
        print(f"Chat {i}: ✅ Valid")
    except ValidationError as e:
        print(f"Chat {i}: ❌ Invalid - {e.message}")


Chat 1: ✅ Valid
Chat 2: ✅ Valid
Chat 3: ✅ Valid


In [42]:

import os
import json
from openai import OpenAI
from jsonschema import validate, ValidationError

# ---------- Groq API Key ----------
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY not found. Set it with `%env GROQ_API_KEY=your_key`")

client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=api_key)

# ---------- JSON Schema ----------
chat_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

# ---------- Sample Chats ----------
sample_chats = [
    "Hi, my name is Vishal Kumar. My email is vishal@example.com, phone 9876543210, I live in Delhi and I am 20 years old.",
    "Hello, I'm Priya Sharma, email priya123@mail.com, phone 9123456789, from Mumbai, 22 years old.",
    "Hey, name is Raj, email raj456@gmail.com, phone 9988776655, living in Bangalore, age 25."
]

# ---------- Function to Extract Info Using Groq ----------
def extract_info_groq(chat_text):
    response = client.chat.completions.create(
        model="groq/compound-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Extract the following info in JSON format exactly matching this schema: "
                    + json.dumps(chat_schema)
                )
            },
            {"role": "user", "content": chat_text}
        ],
        max_tokens=200
    )

    try:
        # Parse API response to JSON
        result_text = response.choices[0].message.content.strip()
        result_json = json.loads(result_text)

        # Validate against schema
        validate(instance=result_json, schema=chat_schema)
        return result_json

    except json.JSONDecodeError:
        # Agar JSON parse fail ho jaye
        return {"error": "Invalid JSON returned", "raw_output": response.choices[0].message.content}

    except ValidationError as ve:
        # Agar schema validate fail ho jaye
        return {"error": f"Schema validation failed: {ve.message}", "raw_output": result_text}

# ---------- Process All Sample Chats ----------
parsed_outputs = [extract_info_groq(chat) for chat in sample_chats]

# ---------- Print Final Structured Outputs ----------
for i, output in enumerate(parsed_outputs, 1):
    print(f"--- Chat {i} Parsed ---")
    print(json.dumps(output, indent=4))


--- Chat 1 Parsed ---
{
    "error": "Invalid JSON returned",
    "raw_output": "Based on the information you provided, here is the extracted data in JSON format:\n\n```\n{\n  \"name\": \"Vishal Kumar\",\n  \"email\": \"vishal@example.com\",\n  \"phone\": \"9876543210\",\n  \"location\": \"Delhi\",\n  \"age\": 20\n}\n```"
}
--- Chat 2 Parsed ---
{
    "error": "Invalid JSON returned",
    "raw_output": "Based on the information you provided, here is the extracted data in JSON format:\n\n```\n{\n  \"name\": \"Priya Sharma\",\n  \"email\": \"priya123@mail.com\",\n  \"phone\": \"9123456789\",\n  \"location\": \"Mumbai\",\n  \"age\": 22\n}\n```"
}
--- Chat 3 Parsed ---
{
    "error": "Invalid JSON returned",
    "raw_output": "Based on the information provided, here is the extracted data in JSON format:\n\n```\n{\n  \"name\": \"Raj\",\n  \"email\": \"raj456@gmail.com\",\n  \"phone\": \"9988776655\",\n  \"location\": \"Bangalore\",\n  \"age\": 25\n}\n```"
}
